In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

import warnings
warnings.filterwarnings('ignore')

C:\Users\Berkay Kaya\Desktop\BEKO_Staj\Proje\ai-assistant\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f"Torch CUDA erişilebilir mi: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Ortamdaki GPU Sayısı: {torch.cuda.device_count()}")
    print(f"Ortamdaki Ana GPU Adı: {torch.cuda.get_device_name(0)}")

Torch CUDA erişilebilir mi: True
Ortamdaki GPU Sayısı: 1
Ortamdaki Ana GPU Adı: NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

#quantization_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_quant_type="nf4",
#    bnb_4bit_compute_dtype=torch.bfloat16
#)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 3/3 [00:07<00:00,  2.54s/it]
Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [5]:
tokenizer.save_pretrained("../Models/tokenizer-mistralai-7b-instruct")
model.save_pretrained("../Models/model-mistralai-7b-instruct")

Saving checkpoint shards: 100%|██████████████████████████████████████████████████████████| 6/6 [04:00<00:00, 40.14s/it]


In [5]:
messages = [
    {
        "role": "user",
        "content": """
Your task is to extract specific fields from the given input sentence and return the result strictly JSON format.

Information:
- Don't confuse theese two words "current" and "last time"


Example:
Input: "What is the current temperature in the living room?"
Output: {"time": "current", "room": "living room", "query": "temperature"}

Example:
Input: "What was the temperature in the bedroom yesterday?"
Output: {"time": "yesterday", "room": "bedroom", "query": "temperature"}

Example:
Input: "What was the temperature in the bedroom last time?"
Output: {"time": "last time", "room": "bedroom", "query": "temperature"}

Example:
Input: "What is the current humidity level in the sitting room?"
Output: {"time": "current", "room": "sitting room", "query": "humidity"}

Example:
Input: "What is the current temperature in the bathroom?"
Output: {"time": "current", "room": "bathroom", "query": "temperature"}

Now process the following sentence:
Input: "What was the temperature in the sitting room last time"
Output:
"""
    }
]

In [6]:
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=150, temperature=0.0)

result = tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True) # sadece modelin ürettiği yeni tokenleri decode et

print(result)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{"time": "last time", "room": "sitting room", "query": "temperature"}


In [7]:
test_messages = [
    {
        "role" : "user",
        "content" : """
Your goal is to answer the user's question using only the information provided to you.  
- If the information is enough, give a clear and complete sentence as the answer, not just a number.  
- If the information is not enough, say that you don’t have information about it.  

Example:
User Question: "What was the temperature in the sitting room yesterday"  
Information: Date: 2025-08-15, Time: 9:00, Room: Sitting room, Sensor: Temperature Sensor, Sensor Value: 23.4, Sensor Unit: °C, Status: Low Temperature  
Output: "The recorded temperature in the sitting room was 23.4°C yesterday, which was considered low."

Now process the following sentence:
User Question: "What is the current temperature in the living room"
Information: Date: 2025-08-16, Time: 15:00, Room: Living Room, Sensor: Temperature Sensor, Sensor Value: 25.4, Sensor Unit: °C, Status: Normal
Output:
"""
    }
]

In [8]:
test_inputs = tokenizer.apply_chat_template(test_messages, return_tensors="pt").to(model.device)
test_outputs = model.generate(test_inputs, max_new_tokens=150, temperature=0)

test_result = tokenizer.decode(test_outputs[0][test_inputs.shape[-1]:], skip_special_tokens=True)

print(test_result)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The current temperature in the living room is 25.4°C, which is considered normal.
